<a href="https://colab.research.google.com/github/ridwanbello/facial-expression-with-deep-learning/blob/master/facial_expression_training_and_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TRAINING, VALIDATION AND TESTING OF PRE-PROCESSED IMAGES

In [1]:
# Importing necessary modules and packages
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report,confusion_matrix

import cv2
import os

import numpy as np

In [2]:
!nvidia-smi

Sat Dec  3 10:55:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    30W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17562835914994826230
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14415560704
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2716693324483333563
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [5]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
!unzip "/content/gdrive/MyDrive/dats.zip"

Archive:  /content/gdrive/MyDrive/dats.zip
   creating: dats/train/
   creating: dats/train/angry/
  inflating: dats/train/angry/angryss441.jpg  
  inflating: dats/train/angry/angryss442.jpg  
  inflating: dats/train/angry/angryss443.jpg  
  inflating: dats/train/angry/angryss444.jpg  
  inflating: dats/train/angry/angryss445.jpg  
  inflating: dats/train/angry/angryss446.jpg  
  inflating: dats/train/angry/angryss447.jpg  
  inflating: dats/train/angry/angryss448.jpg  
  inflating: dats/train/angry/angryss449.jpg  
  inflating: dats/train/angry/angryss450.jpg  
  inflating: dats/train/angry/angryss451.jpg  
  inflating: dats/train/angry/angryss452.jpg  
  inflating: dats/train/angry/angryss453.jpg  
  inflating: dats/train/angry/angryss454.jpg  
  inflating: dats/train/angry/angryss455.jpg  
  inflating: dats/train/angry/angryss456.jpg  
  inflating: dats/train/angry/angryss457.jpg  
  inflating: dats/train/angry/angryss458.jpg  
  inflating: dats/train/angry/angryss459.jpg  
  inflat

In [8]:
INIT_LR = 1e-4
BS = 16
NUM_EPOCHS = 20
CLASSES=["Angry","Fear","Happy","Sad","Surprise"]

In [9]:
#Now we can easily fetch our train and validation data.
import os
BASE_PATH = "/content/dats"
TRAIN_PATH = os.path.sep.join([BASE_PATH, "train"])
VAL_PATH = os.path.sep.join([BASE_PATH, "val"])

In [11]:
totalTrain = len(list(paths.list_images(TRAIN_PATH)))
totalVal = len(list(paths.list_images(VAL_PATH)))

In [12]:
# initialize the training training data augmentation object
trainAug = ImageDataGenerator(
	#rescale=1./255,
	rotation_range=25,
	zoom_range=0.2,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.2,
	horizontal_flip=True,
	fill_mode="nearest")
# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()
# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [13]:
# initialize the training generator
trainGen = trainAug.flow_from_directory(
	TRAIN_PATH,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=True,
	batch_size=BS)


# initialize the validation generator
valGen = valAug.flow_from_directory(
	VAL_PATH,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)

Found 2273 images belonging to 5 classes.
Found 572 images belonging to 5 classes.


In [14]:
X_train, y_train = next(trainGen)

In [15]:
X_test, y_test = next(valGen)

In [16]:
X_train.shape

(16, 224, 224, 3)

In [17]:
totalTrain = len(list(paths.list_images(TRAIN_PATH)))
totalVal = len(list(paths.list_images(VAL_PATH)))

Install VGGFACE2 pre-built model for training

In [18]:
!pip install git+https://github.com/rcmalli/keras-vggface.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-8k50ka8d
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-8k50ka8d
  Created wheel for keras-vggface: filename=keras_vggface-0.6-py3-none-any.whl size=8325 sha256=a3c2f9f4ed0f9def0a2516a13615d1bc4efeaccf043e823518b704bbb434d901
  Stored in directory: /tmp/pip-ephem-wheel-cache-oo7ndeu3/wheels/24/de/3d/9a0fad8c64abbc63b7fb8ddcdf1aa0a4c04a98d7936e989a29
Successfully built keras-vggface


In [19]:
!pip install keras_vggface


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
!pip install keras_applications

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
!pip install keras_applications --no-deps
filename = "/usr/local/lib/python3.8/dist-packages/keras_vggface/models.py"
text = open(filename).read()
open(filename, "w+").write(text.replace('keras.engine.topology', 'tensorflow.keras.utils'))


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
from keras_vggface.vggface import VGGFace
from keras import Model
from keras.layers import Flatten, Dense, Input

In [27]:
nb_class = 5

vgg_model = VGGFace(model='resnet50',include_top=False, input_shape=(224, 224, 3))
last_layer = vgg_model.get_layer('avg_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
out = Dense(nb_class, activation='softmax', name='classifier')(x)
custom_vgg_model = Model(vgg_model.input, out)
for layer in vgg_model.layers:
	layer.trainable = False

94694792/94694792 [==============================] - 2s 0us/step


In [28]:
# compile the model
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / NUM_EPOCHS)
custom_vgg_model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the model
print("[INFO] training model...")
H = custom_vgg_model.fit_generator(
	generator=trainGen,
	steps_per_epoch=totalTrain // BS,
	validation_data=valGen,
	validation_steps=totalVal // BS,
	epochs=NUM_EPOCHS)

[INFO] training model...


<ipython-input-28-d0a13b901ad1>:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  H = custom_vgg_model.fit_generator(


Epoch 1/20
142/142 [==============================] - 45s 244ms/step - loss: 2.7237 - accuracy: 0.3744 - val_loss: 1.0051 - val_accuracy: 0.6339
Epoch 2/20
142/142 [==============================] - 29s 203ms/step - loss: 1.3377 - accuracy: 0.5441 - val_loss: 0.8272 - val_accuracy: 0.6964
Epoch 3/20
142/142 [==============================] - 27s 193ms/step - loss: 0.9985 - accuracy: 0.6225 - val_loss: 0.7208 - val_accuracy: 0.7143
Epoch 4/20
142/142 [==============================] - 28s 194ms/step - loss: 0.8797 - accuracy: 0.6690 - val_loss: 0.6841 - val_accuracy: 0.7304
Epoch 5/20
142/142 [==============================] - 28s 195ms/step - loss: 0.7981 - accuracy: 0.6945 - val_loss: 0.6330 - val_accuracy: 0.7625
Epoch 6/20
142/142 [==============================] - 28s 194ms/step - loss: 0.7442 - accuracy: 0.7226 - val_loss: 0.5870 - val_accuracy: 0.7821
Epoch 7/20
142/142 [==============================] - 29s 202ms/step - loss: 0.6809 - accuracy: 0.7514 - val_loss: 0.5612 - val_ac

In [29]:
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")

In [30]:
%matplotlib inline

In [31]:

!mkdir -p nisfe_model
custom_vgg_model.save('nisfe_model/my_model') 

In [32]:
new_model = tf.keras.models.load_model('nisfe_model/my_model')

# Check its architecture
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1/7x7_s2 (Conv2D)          (None, 112, 112, 64  9408        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv1/7x7_s2/bn (BatchNormaliz  (None, 112, 112, 64  256        ['conv1/7x7_s2[0][0]']           
 ation)                         )                                                             

In [33]:
!zip -r model.zip /content/nisfe_model

  adding: content/nisfe_model/ (stored 0%)
  adding: content/nisfe_model/my_model/ (stored 0%)
  adding: content/nisfe_model/my_model/keras_metadata.pb (deflated 95%)
  adding: content/nisfe_model/my_model/variables/ (stored 0%)
  adding: content/nisfe_model/my_model/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/nisfe_model/my_model/variables/variables.index (deflated 79%)
  adding: content/nisfe_model/my_model/assets/ (stored 0%)
  adding: content/nisfe_model/my_model/saved_model.pb (deflated 92%)


In [34]:
try:
  from google.colab import files
  files.download('./model.zip')
except ImportError:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

EVALUATION SECTION

In [35]:
test_generator = ImageDataGenerator()
test_data_generator = test_generator.flow_from_directory(
    VAL_PATH, # Put your path here
     target_size=(224, 224),
    batch_size=16,
    shuffle=False)
test_steps_per_epoch = np.math.ceil(test_data_generator.samples / test_data_generator.batch_size)

predictions = custom_vgg_model.predict(test_data_generator, steps=test_steps_per_epoch)
# Get most likely class
predicted_classes = np.argmax(predictions, axis=1)

Found 572 images belonging to 5 classes.
36/36 [==============================] - 3s 63ms/step


In [36]:
true_classes = test_data_generator.classes
class_labels = list(test_data_generator.class_indices.keys())

predicted_classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 3, 1, 0, 3, 4, 3, 0, 3, 2, 0, 3, 3, 3, 3, 3, 0,
       3, 3, 3, 3, 1, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 3, 4, 3, 3, 2, 2, 2, 3, 3, 3, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 3, 2, 2, 2, 2, 3, 2, 2, 0, 2, 2, 4, 2, 2, 2,
       2, 4, 2, 4, 3, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2,
       3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [37]:
from sklearn.metrics import classification_report, accuracy_score
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

              precision    recall  f1-score   support

       angry       0.91      0.83      0.87       117
        fear       0.91      0.84      0.88       109
       happy       0.81      0.91      0.86       115
         sad       0.76      0.81      0.78       119
    surprise       0.90      0.87      0.88       112

    accuracy                           0.85       572
   macro avg       0.86      0.85      0.85       572
weighted avg       0.86      0.85      0.85       572



# Hyperparameters Used with VGGFace Model
## Pre-trained Weights: ResNet50
## Epochs: 20
## Optimizer: Adam
## Learning Rate: 0.0001
## Training Accuracy: 0.86
## Training Loss: 0.36
## Validation Accuracy: 0.84
## Validation Loss: 0.43


Testing with FER-2013 test dataset

In [51]:
!unzip "/content/gdrive/MyDrive/fers-test.zip"


Archive:  /content/gdrive/MyDrive/fers-test.zip
   creating: fers-test/val/
   creating: fers-test/val/angry/
 extracting: fers-test/val/angry/10.jpg  
 extracting: fers-test/val/angry/101.jpg  
 extracting: fers-test/val/angry/103.jpg  
 extracting: fers-test/val/angry/111.jpg  
 extracting: fers-test/val/angry/115.jpg  
 extracting: fers-test/val/angry/119.jpg  
 extracting: fers-test/val/angry/125.jpg  
 extracting: fers-test/val/angry/126.jpg  
 extracting: fers-test/val/angry/142.jpg  
 extracting: fers-test/val/angry/147.jpg  
 extracting: fers-test/val/angry/1487.jpg  
 extracting: fers-test/val/angry/149.jpg  
 extracting: fers-test/val/angry/1492.jpg  
 extracting: fers-test/val/angry/1500.jpg  
 extracting: fers-test/val/angry/1504.jpg  
 extracting: fers-test/val/angry/1505.jpg  
 extracting: fers-test/val/angry/1514.jpg  
 extracting: fers-test/val/angry/1518.jpg  
 extracting: fers-test/val/angry/1524.jpg  
 extracting: fers-test/val/angry/1541.jpg  
 extracting: fers-test

In [52]:
BASE_PATH = "/content/fers-test"


In [53]:
TEST_PATH = os.path.sep.join([BASE_PATH, "val"])
totalTest = len(list(paths.list_images(TEST_PATH)))

In [54]:
totalTest

551

In [55]:

fertest_generator = ImageDataGenerator()
fertest_data_generator = test_generator.flow_from_directory(
    TEST_PATH, # Put your path here
     target_size=(224, 224),
    batch_size=16,
    shuffle=False)
fertest_steps_per_epoch = np.math.ceil(fertest_data_generator.samples / fertest_data_generator.batch_size)

ferpredictions = custom_vgg_model.predict(fertest_data_generator, steps=fertest_steps_per_epoch)
# Get most likely class
ferpredicted_classes = np.argmax(ferpredictions, axis=1)

Found 551 images belonging to 5 classes.
35/35 [==============================] - 2s 59ms/step


In [56]:
fertrue_classes = fertest_data_generator.classes
ferclass_labels = list(fertest_data_generator.class_indices.keys())
from sklearn.metrics import classification_report, accuracy_score
report = classification_report(fertrue_classes, ferpredicted_classes, target_names=class_labels)
print(report)

              precision    recall  f1-score   support

       angry       0.41      0.21      0.28       111
        fear       0.17      0.09      0.12       110
       happy       0.39      0.79      0.52       110
         sad       0.27      0.42      0.33       110
    surprise       0.55      0.21      0.30       110

    accuracy                           0.34       551
   macro avg       0.36      0.34      0.31       551
weighted avg       0.36      0.34      0.31       551

